<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_Assignment1/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
!wandb login

In [ ]:
import keras
from keras.datasets import fashion_mnist
from PIL import Image
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

wandb.init(project="assignment-1", entity="swathi")
classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

In [3]:
def load_images():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  X_train_img = x_train[:54000]
  X_validation_img = x_train[54000:]
  X_test_img = x_test
  return X_train_img, X_validation_img, X_test_img

In [61]:
def load_data():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  X_train = np.transpose((np.array(x_train[:54000],dtype=int).reshape([54000,784]))/4)
  Y_train = (np.array(y_train[:54000],dtype=int).reshape([54000,1]))
  X_validation = np.transpose((np.array(x_train[54000:],dtype=int).reshape([6000,784]))/4)
  Y_validation = np.array(y_train[54000:],dtype=int).reshape([6000,1])
  X_test = np.transpose((np.array(x_test,dtype=int).reshape([10000,784]))/4)
  Y_test = np.array(y_test,dtype=int).reshape([10000,1])
  return X_train, Y_train, X_validation, Y_validation, X_test, Y_test

In [39]:
def log_images():
  images = []
  labels = []
  for i in range(len(X_train)):
    if len(labels)==len(classes):
      break
    j = random.randint(0,53999)
    if classes[Y_train[j]] not in labels:
      images.append(X_train_img[j])
      labels.append(classes[Y_train[j]])
  wandb.log({"Examples": [wandb.Image(img, caption=caption) for img,caption in zip(images,labels)]})

Loading data

In [53]:
X_train, Y_train, X_validation, Y_validation, X_test, Y_test = load_data()
X_train_img, X_validation_img, X_test_img = load_images()

In [8]:
log_images()

Weights initialising

In [62]:
L = 5 #Total number of layers = input, output + hidden
N = np.zeros(L, dtype = int) # Number of neurons in each layer
N[0] = X_train.shape[1] #Number of parameters
N[1] = 32
N[2] = 32
N[3] = 32
N[4] = 10
W = []
grad_W=[]
b = []
grad_b=[]
a=[]
grad_a=[]
a.append(0)
h=[]
grad_h=[]
h.append(X_train)
np.random.seed(0)

In [63]:
def initialize_weights(N):
  W.append(0)
  b.append(0)
  for i in range(1,L+1):
    W.append(np.random.randn(N[i],N[i-1]))
    b.append(np.random.randn(N[1],1))
  return W,b 

In [64]:
def one_hot(y):
  one_hot_y=[]
  for i in range(N[0]):
    one_hot=np.zeros((10,1))
    one_hot[y[i]-1] = 1
    one_hot_y.append(one_hot)
  
  return one_hot_y

Softmax

In [13]:
def softmax(a):
  return np.exp(a)/sum(np.exp(a))

Sigmoid

In [14]:
def sigmoid(a):

  return 1/(1+np.exp(-a))


Sigmoid derivative

In [15]:
def dSigmoid(a):

  return sigmoid(a)*(1-sigmoid(a))

Feed forward network

In [58]:
def feedforward(X_input, W, b, N, l):
  h_prev=X_input
  for i in range(1,l) :
    a_new=b[i]+np.matmul(W[i],h_prev)
    h=sigmoid(a_new)
    a.append(a_new)
    h.append(h)
    h_prev=h

  a_new=b[l]+np.matmul(W[l],h_prev)
  y_hat=softmax(a_new)
  a.append(a_new)

  return a,h,y_hat

Backpropogation

In [35]:
def backrop(y,y_hat,W,a,h):
  da = y_hat - one_hot(y)
  grad_a.append(da)

  for i in reversed(range(1,L+1)):
    dw = np.matmul(da,np.transpose(h[i-1]))
    db = da
    dh_prev = np.matmul(np.transpose(W[i]),da)
    da_prev = dh_prev*dSigmoid(a[i-1])
    
    grad_W.append(dw)
    grad_b.append(db)
    grad_a.append(da_prev)
    grad_h.append(dh_prev)

  return grad_W,grad_b,grad_a,grad_h


Gradient Descent

In [20]:
def grad_descent(N,l,X_input,y,opt):
  if opt == 'sgd' :
    W = W - grad_W
    b = b - grad_b

Training

In [25]:
def training(N,l,opt,epochs,X_input,y):
  
  W , b = initialize_weights(N)
  for i in range(epochs):
    a , h , y_hat = feedforward(X_input, W, b, N, l)  
    grad_W , grad_b , _ , _ = backrop(y,y_hat,W,a,h)
    W , b = grad_descent(N,l,X_train,opt)

  return W,b


In [24]:
def predict(W,b,X_test,N,l):
  _, _, y_hat = feedforward(X_test, W, b, N, l)
  y_class = np.argmax(y_hat,axis=0)

  return y_class

In [ ]:
feedforward (X_train, W, b, N, L)